### We are going to predict whether the note is Fake or Real

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('BankNote_Authentication.csv')
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [3]:
df.shape

(1372, 5)

In [4]:
df.describe()

,variance,skewness,curtosis,entropy,class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [9]:
df.corr()['class']

variance   -0.724843
skewness   -0.444688
curtosis    0.155883
entropy    -0.023424
class       1.000000
Name: class, dtype: float64

In [11]:
X =df.drop('class',axis=1) 
y = df['class']

In [12]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y,train_size=0.9,random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [15]:
import keras_tuner
import keras

In [16]:
def build_model(hp):
  model = keras.Sequential()
  
  model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32,64,128]),
      hp.Choice("activation ", ['relu','sigmoid']))
       )
  
  model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32,64,128]),
      activation=hp.Choice('activation', ['relu', 'sigmoid'])))
  
  model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32,64,128]),
      activation=hp.Choice('activation', ['relu', 'sigmoid'])))
  
  
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy')
  
  return model

In [17]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

In [18]:
tuner.search(X_train, y_train, epochs=5, validation_split=0.2)
best_model = tuner.get_best_models()[0]

Trial 5 Complete [00h 00m 03s]
val_loss: 0.558409571647644

Best val_loss So Far: 0.008275861851871014
Total elapsed time: 00h 00m 16s


d:\Ann prac-2\venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]


In [20]:
best_hyperparameters = best_trial.hyperparameters.values


In [21]:
print("Best Hyperparameters:")
for key, value in best_hyperparameters.items():
    print(f"{key}: {value}")

# Optionally, get the best model
best_model = tuner.get_best_models()[0]

# Summary of the best model
best_model.summary()

Best Hyperparameters:
units: 32
activation : relu
activation: relu


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,305 (9.00 KB)

 Trainable params: 2,305 (9.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:

# Now, get the best model
best_model = tuner.get_best_models()[0]

# Inspect the layers of the best model and print out the hyperparameters for each layer
print("\nBest Model's Layer Details:")
for layer in best_model.layers:
    if isinstance(layer, keras.layers.Dense):
        print(f"Layer: {layer.name}")
        print(f"  - Units: {layer.units}")
        print(f"  - Activation: {layer.activation.__name__}")
        # print(f"  - Input shape: {layer.input_shape}")
        # print(f"  - Output shape: {layer.output_shape}")
        print()


Best Model's Layer Details:
Layer: dense
  - Units: 32
  - Activation: relu

Layer: dense_1
  - Units: 32
  - Activation: relu

Layer: dense_2
  - Units: 32
  - Activation: relu

Layer: dense_3
  - Units: 1
  - Activation: sigmoid



In [23]:
# Get the best model after hyperparameter tuning
best_model = tuner.get_best_models()[0]

# Compile the best model (if necessary)
best_model.compile(optimizer='adam',  # You can use 'adam' or the optimizer that was selected during tuning
                   loss='binary_crossentropy',  # Assuming a binary classification problem (diabetes prediction)
                   metrics=['accuracy', 'recall'])

# Now, train the best model
history = best_model.fit(X_train, y_train, 
                         epochs=10,  # Choose the number of epochs
                         batch_size=32,  # You can adjust the batch size
                        #  validation_data=(X_val, y_val),  # Optional: Use validation data if available
                        #  class_weight={0: 1, 1: 5})  # Adjust class_weight if dealing with imbalanced classes
)



Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 0.0082 - recall: 1.0000
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0027 - recall: 1.0000   
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0012 - recall: 1.0000
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 8.5345e-04 - recall: 1.0000
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 4.8164e-04 - recall: 1.0000
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 4.0590e-04 - recall: 1.0000
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 2.4313e-04 - recall: 1.0000
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 2.3934e-04 - recall: 1.0000
Epoch 9/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.4348e-04 - recall: 1.0000
Epoch 10/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1

In [24]:
y_pred = best_model.predict(X_test)
y_pred = (y_pred>0.5).astype(int)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [29]:
from sklearn.metrics import confusion_matrix , classification_report ,precision_score , recall_score

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        77
           1       1.00      1.00      1.00        61

    accuracy                           1.00       138
   macro avg       1.00      1.00      1.00       138
weighted avg       1.00      1.00      1.00       138



In [27]:
confusion_matrix(y_test , y_pred)

array([[77,  0],
       [ 0, 61]])

In [30]:
recall_score(y_test,y_pred)

1.0

In [31]:
precision_score(y_test,y_pred)

1.0

In [32]:
import pickle

# Save the model
with open('best_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)

# Save the scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


In [39]:
df[df["class"] == 1]

,variance,skewness,curtosis,entropy,class
762,-1.39710,3.31910,-1.392700,-1.99480,1
763,0.39012,-0.14279,-0.031994,0.35084,1
764,-1.66770,-7.15350,7.892900,0.96765,1
765,-3.84830,-12.80470,15.682400,-1.28100,1
766,-3.56810,-8.21300,10.083000,0.96765,1
...,...,...,...,...,...
1367,0.40614,1.34920,-1.450100,-0.55949,1
1368,-1.38870,-4.87730,6.477400,0.34179,1
1369,-3.75030,-13.45860,17.593200,-2.77710,1
1370,-3.56370,-8.38270,12.393000,-1.28230,1
